In [1]:
import pandas as pd
import pickle

data1 = pd.read_pickle('dtm_stop.pkl')
data1

,ab,abandoned,abano,abilities,able,ables,abm,abms,abraham,abroad,...,yushchenko,zealous,zeffirellis,zenith,zero,zeroed,zled,zone,zoom,zooms
A,0,1,0,0,6,0,6,2,0,2,...,4,0,0,0,1,1,0,1,0,0
B,0,0,0,0,0,0,0,0,0,2,...,0,1,0,0,0,0,0,0,0,0
C,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,4,0,1,0,2,1
D,0,0,1,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
E,0,0,0,1,5,1,0,0,2,0,...,0,0,1,0,0,0,0,0,0,0
F,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [2]:
conda install -c conda-forge gensim

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.




==> WARNING: A newer version of conda exists. <==
  current version: 22.11.1
  latest version: 23.5.2

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.5.2




In [2]:
tdm = data1.transpose()
tdm.head()

,A,B,C,D,E,F
ab,0,0,1,0,0,0
abandoned,1,0,0,0,0,0
abano,0,0,0,1,0,0
abilities,0,0,0,0,1,0
able,6,0,0,2,5,1


In [3]:
from gensim import matutils, models
import scipy.sparse

In [4]:
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [ ]:
cv = pickle.load(open("cv_stop.pkl", "rb"))
id2word = dict((v, k) for k, v in cv.vocabulary_.items())


In [ ]:
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=2, passes=10)
lda.print_topics()

In [ ]:
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=3, passes=10)
lda.print_topics()

In [ ]:
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=4, passes=10)
lda.print_topics()

In [ ]:
from nltk import word_tokenize, pos_tag

def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)

In [5]:
data_clean = pd.read_pickle('data_clean.pkl')
data_clean

,transcript
A,oliver stone conducted interviews with vladimi...
B,by william wolf ingmar bergman stands outside ...
C,alfred hitchcock and saboteur it was and alfr...
D,starlog interview director of dune the filmmak...
E,by jon burlingame and gary crowdus ennio morri...
F,by leonard quart pauline kael shook up the cri...


In [6]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Sahil\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Sahil\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [8]:
data_nouns = pd.DataFrame(data_clean.transcript.apply(nouns))
data_nouns

NameError: name 'nouns' is not defined

In [ ]:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

# Re-add the additional stop words since we are recreating the document-term matrix
add_stop_words = ['like', 'im', 'know', 'just', 'dont', 'thats', 'right', 'people',
                  'youre', 'got', 'gonna', 'time', 'think', 'yeah', 'said']
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

# Recreate a document-term matrix with only nouns
cvn = CountVectorizer(stop_words=stop_words)
data_cvn = cvn.fit_transform(data_nouns.transcript)
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names())
data_dtmn.index = data_nouns.index
data_dtmn

In [ ]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())

In [ ]:
# Let's start with 2 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=2, id2word=id2wordn, passes=10)
ldan.print_topics()

In [ ]:
# Let's try topics = 3
ldan = models.LdaModel(corpus=corpusn, num_topics=3, id2word=id2wordn, passes=10)
ldan.print_topics()

In [ ]:
# Let's try 4 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=4, id2word=id2wordn, passes=10)
ldan.print_topics()

In [ ]:
def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)] 
    return ' '.join(nouns_adj)

In [ ]:
data_nouns_adj = pd.DataFrame(data_clean.transcript.apply(nouns_adj))
data_nouns_adj

In [ ]:
cvna = CountVectorizer(stop_words=stop_words, max_df=.8)
data_cvna = cvna.fit_transform(data_nouns_adj.transcript)
data_dtmna = pd.DataFrame(data_cvna.toarray(), columns=cvna.get_feature_names())
data_dtmna.index = data_nouns_adj.index
data_dtmna

In [ ]:
# Create the gensim corpus
corpusna = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmna.transpose()))

# Create the vocabulary dictionary
id2wordna = dict((v, k) for k, v in cvna.vocabulary_.items())

In [ ]:
# Let's start with 2 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=2, id2word=id2wordna, passes=10)
ldana.print_topics()

In [ ]:
# Let's try 3 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=3, id2word=id2wordna, passes=10)
ldana.print_topics()

In [ ]:
# Let's try 4 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=10)
ldana.print_topics()

In [ ]:
# Our final LDA model (for now)
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=80)
ldana.print_topics()

In [ ]:
# Let's take a look at which topics each transcript contains
corpus_transformed = ldana[corpusna]
list(zip([a for [(a,b)] in corpus_transformed], data_dtmna.index))


In [ ]:
ldana = models.LdaModel(corpus=corpusna, num_topics=5, id2word=id2wordna, passes=80)
ldana.print_topics()

In [ ]:
ldana = models.LdaModel(corpus=corpusna, num_topics=6, id2word=id2wordna, passes=80)
ldana.print_topics()

In [ ]:
ldana = models.LdaModel(corpus=corpusna, num_topics=5, id2word=id2wordna, passes=100)
ldana.print_topics()

In [ ]:
# Let's take a look at which topics each transcript contains
corpus_transformed = ldana[corpusna]
list(zip([a for [(a,b)] in corpus_transformed], data_dtmna.index))

In [ ]:
def nouns_verb(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and Verb.'''
    is_noun_verb = lambda pos: pos[:2] == 'NN' or pos[:2]=='VB'
    tokenized = word_tokenize(text)
    nouns_verb = [word for (word, pos) in pos_tag(tokenized) if is_noun_verb(pos)] 
    return ' '.join(nouns_verb)

In [ ]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns_verb = pd.DataFrame(data_clean.transcript.apply(nouns_verb))
data_nouns_verb

In [ ]:
# Create a new document-term matrix using only nouns
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

# Re-add the additional stop words since we are recreating the document-term matrix
add_stop_words = ['like', 'im', 'know', 'just', 'dont', 'thats', 'right', 'people',
                  'youre', 'got', 'gonna', 'time', 'think', 'yeah', 'said']
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

# Create a new document-term matrix using only nouns and adjectives, also remove common words with max_df
cvnv = CountVectorizer(stop_words=stop_words, max_df=.8)
data_cvnv = cvnv.fit_transform(data_nouns_verb.transcript)
data_dtmnv = pd.DataFrame(data_cvnv.toarray(), columns=cvnv.get_feature_names())
data_dtmnv.index = data_nouns_verb.index
data_dtmnv

In [ ]:
# Create the gensim corpus
corpusnv = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmnv.transpose()))

# Create the vocabulary dictionary
id2wordnv = dict((v, k) for k, v in cvnv.vocabulary_.items())

In [ ]:
# Let's start with 2 topics
ldanv = models.LdaModel(corpus=corpusnv, num_topics=2, id2word=id2wordnv, passes=10)
ldanv.print_topics()

In [ ]:
# Let's start with 3 topics
ldanv = models.LdaModel(corpus=corpusnv, num_topics=3, id2word=id2wordnv, passes=10)
ldanv.print_topics()

In [ ]:
# Let's start with 4 topics
ldanv = models.LdaModel(corpus=corpusnv, num_topics=4, id2word=id2wordnv, passes=10)
ldanv.print_topics()

In [ ]:
# Our final LDA model (for now)
ldanv = models.LdaModel(corpus=corpusnv, num_topics=4, id2word=id2wordnv, passes=80)
ldanv.print_topics()

In [ ]:
# Let's take a look at which topics each transcript contains
corpus_transformed = ldanv[corpusnv]
list(zip([a for [(a,b)] in corpus_transformed], data_dtmnv.index))